In [2]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_130347/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [4]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [6]:
analize = Analizer(0.7)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_4_8_9,0.994993,0.991621,0.983928,0.987243,1.474148,1.721520,5.530713,3.514079,"Hidden Size=[25], regularizer=0.1, learning_ra..."
1,model_4_8_10,0.994983,0.990987,0.982717,0.986281,1.477147,1.851791,5.947575,3.779216,"Hidden Size=[25], regularizer=0.1, learning_ra..."
2,model_4_8_8,0.994955,0.992282,0.985200,0.988252,1.485517,1.585625,5.093194,3.236243,"Hidden Size=[25], regularizer=0.1, learning_ra..."
3,model_4_8_11,0.994938,0.990385,0.981573,0.985371,1.490490,1.975390,6.341133,4.029859,"Hidden Size=[25], regularizer=0.1, learning_ra..."
4,model_4_8_12,0.994868,0.989819,0.980501,0.984517,1.511096,2.091727,6.710152,4.265099,"Hidden Size=[25], regularizer=0.1, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...
179,model_4_7_2,0.970703,0.980525,0.992163,0.986446,8.626116,4.276708,2.004744,3.207547,"Hidden Size=[25], regularizer=0.1, learning_ra..."
226,model_4_7_1,0.964989,0.981216,0.994095,0.987768,10.308460,4.124982,1.510639,2.894699,"Hidden Size=[25], regularizer=0.1, learning_ra..."
247,model_4_7_0,0.957740,0.981035,0.995753,0.988523,12.442671,4.164666,1.086330,2.716030,"Hidden Size=[25], regularizer=0.1, learning_ra..."
255,model_3_7_8,0.950865,0.938353,0.982916,0.959998,14.467010,28.897543,7.985468,19.056568,"Hidden Size=[25], regularizer=0.01, learning_r..."
